# Intro

This notebook provides a complete "edge-ready" workflow for performing object detection on high-resolution satellite imagery using limited hardware resources (CPU only). It guides users through setting up the environment, downloading optical imagery of the Port of Rotterdam, and applying a lightweight YOLOv8 model to detect objects within large images using slicing techniques. Finally, the notebook demonstrates how to visualize these detections.

It is designed to be a partial solution, leaving hackathon participants plenty of opportunity for scaling and optimizing.

The evaluation will look at the inference time. In this notebook this is the "Running the Model" section. If you use preprocessing, include this in the inference block so that the execution time remains accurate. We will also evaluate the interpretation and display of results, whether in this notebook, a seperate dashboard, or a static image. Consider the evaluation a balance of speed and utility (your "So what" with the resulting detection).

For a video walkthrough: https://drive.google.com/file/d/1LIg6HTszGtkUfZqGDlh2vc3AIAbWwN0p/view?usp=sharing 

# Set-up

In [12]:
import sys
sys.executable

'/opt/homebrew/Cellar/jupyterlab/4.5.0_1/libexec/bin/python'

In [13]:
# Install packages from requirements.txt
!pip3 install -r requirements.txt -q

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [14]:
# Standard Library Imports
import os
import glob
import sys

# Fix OpenCV import issues in headless environments
# Set environment variables to prevent GUI-related errors
os.environ['QT_QPA_PLATFORM'] = 'offscreen'

# Make sure src is in path if needed
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

# Project Imports
from src.utils import setup_logging, inspect_environment
from src.config import LOCAL_DIR
from src.data_loader import list_s3_files, download_files_parallel
from src.model_handler import load_model
from src.inference import run_inference
from src.visualization import plot_results, create_map, create_map_by_day

ModuleNotFoundError: No module named 'torch'

In [ ]:
# Adjust logging to be minimal
setup_logging()

# Intro to the Workspace
The goal of the challenge is to use a more constrained hardware with limited CPU cores and no GPU.

In [ ]:
# Overview of the VM
inspect_environment()

# Download the Data
We will use the Port of Rotterdam and Pan-Sharpened RGB imagery from WorldView-2 with a resolution of 0.5m. Note that this dataset will have some image gaps.  




In [ ]:
# 1. Configuration & Download
tasks = list_s3_files()
download_files_parallel(tasks)

# Running the Model -- The Speed Evaluation Block
We use a pre-trained YOLOv8 Nano model to demonstrate immediate object detection on CPU for select classes. If you choose to train your own model using the labels found in spacenet/SN6_buildings/train/AOI_11_Rotterdam/summaryData/, we recommend prioritizing lightweight architectures to focus on speed. As we do not provide a GPU, we do not expect teams to train models. But we do encourage looking for other pretrained models that may be more accurate.

**Note:** The inference pipeline automatically detects and crops black bars from images before processing. This preprocessing step improves detection accuracy by removing edge artifacts that can interfere with object detection. The black bar detection uses a threshold-based approach to identify and remove uniform black regions at image borders.

In [ ]:
# 1. Setup Model
detection_model = load_model()

In [ ]:
# EVALUATION BLOCK.

# 2. Setup Data
folder = LOCAL_DIR
files = sorted(glob.glob(os.path.join(folder, "*.tif")))[:10]

# Define a simple analytics callback for demonstration
def print_detection_count(path, result, img, transform):
    count = len(result.object_prediction_list)
    if count > 0:
        # print(f"Image {os.path.basename(path)}: Found {count} objects")
        pass

# 3. Process Loop (Inference Only)
geo_detections, inference_cache = run_inference(files, detection_model, analytics_callbacks=[print_detection_count])

# Visualize the Results
We use a simple plot to showcase bounding boxes, labels, and accuracy.

In [ ]:
# Select the first 10 file paths to visualize
# The plot_results function now supports optional parameters:
# - max_plots: Maximum number of images to plot (default: 10)
# - output_file: Path to save the plot (default: "detection_plots.png")
plot_results(files, inference_cache, max_plots=10, output_file="detection_plots.png")

# More Advanced Visualization -- The Insights Evaluation Block
We will demo a basic map with the original images and added bounding boxes. Your output can be in any format you like and it does not need to come from a notebook. Just be sure the evaluation team can access it.

The visualization system now supports creating day-specific maps, which can help analyze temporal patterns in detections across different dates.

# Day-Specific Maps
For datasets spanning multiple days, you can create separate maps for each day. This helps analyze temporal patterns and makes it easier to navigate large datasets. The function automatically groups images by date extracted from filenames and creates individual HTML maps for each day.

In [ ]:
# Create day-specific maps
# This will create separate HTML files for each day found in the dataset
# Returns a dictionary mapping day strings (YYYY-MM-DD) to file paths
day_to_filepath = create_map_by_day(files, geo_detections)

# Display the mapping
print("Day-specific maps created:")
for day, filepath in day_to_filepath.items():
    print(f"  {day}: {filepath}")

# Optionally, you can open a specific day's map
# Uncomment the lines below and replace with a specific day if you want to display it
# from IPython.display import HTML
# if day_to_filepath:
#     first_day = list(day_to_filepath.keys())[0]
#     with open(day_to_filepath[first_day], 'r') as f:
#         HTML(f.read())

In [ ]:
# Create combined map with all detections
# The create_map function now supports an optional output_file parameter
m = create_map(files, geo_detections, output_file="map.html")
m

# Helpful Hints
To elevate your solution from a simple detection model to a comprehensive situational awareness tool, consider the following areas:

**1. Preprocessing and managing compute**
Is there a faster way to discard images unlikely to have and desired objects? What steps could help reduce the total inference needed? Any off-the-shelf masking tools available? How do you work around RAM bottlenecks?

**2. Data Enrichment**
Don't rely on imagery alone. Augment your detections with external datasets to provide context:
OpenStreetMap (OSM): Use road networks to determine mobility corridors or building footprints to validate detection accuracy.
Elevation Data (DEM/DSM): Incorporate terrain height to analyze vantage points or flood risks in low-lying areas like Rotterdam.
Weather History: Does weather play a role?


**3. Visualization Strategy**
High-resolution satellite imagery is heavy. How will your end-user interact with it seamlessly?
Implement Image Pyramiding and Tiling to ensure smooth zooming and panning.
Consider converting outputs to Cloud Optimized GeoTIFFs (COGs) for efficient streaming.
Focus on the "Level of Detail" show cluster markers at high altitudes and specific bounding boxes only when zoomed in.


**4. Tactical Utility (The "So What?")**
Analyze the spatial arrangement of detected objects to identify critical lines of sight, cover availability, concealment, and mobility corridors that inform both defensive positioning and offensive maneuvering. Are there military equipment specs that could be combined with the results for a more actionable plan or risk evaluation?
